In [1]:
# import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

import spacy
from spacy.lang.en import English
EN = spacy.load('en_core_web_sm')

In [2]:
data = pd.read_csv('data/Preprocessed_data.csv')

In [27]:
data.head(10)

,original_title,post_corpus,question_content,question_url,tags,overall_scores,answers_content,sentiment_polarity,sentiment_subjectivity,processed_title
0,How can I create a directly-executable cross-p...,create directly executable cross platform gui ...,create directly executable cross platform gui ...,https://stackoverflow.com/questions/2933,python|user-interface|deployment|tkinter|relea...,0.041733,"There's also PyGTK, which is basically a Pytho...",0.172751,0.455014,create directly executable cross platform gui ...
1,Replacement for for... if array iteration,replacement array iteration love list comprehe...,replacement array iteration love list comprehe...,https://stackoverflow.com/questions/13396,.net|python|arrays|loops|iteration,0.000176,in Ruby:,0.038933,0.533347,replacement array iteration
2,Python Regular Expressions to implement string...,python regular expressions implement string un...,python regular expressions implement string un...,https://stackoverflow.com/questions/13791,python|regex|backreference,0.000750,"Well, I think you might have missed the r or m...",-0.122980,0.576632,python regular expressions implement string un...
3,Unicode vs UTF-8 confusion in Python / Django?,unicode vs utf8 confusion python django stumbl...,unicode vs utf8 confusion python django stumbl...,https://stackoverflow.com/questions/22149,python|django|unicode,0.005457,Python stores Unicode as UTF-16. str() will re...,0.082857,0.403250,unicode vs utf8 confusion python django
4,Django ImageField core=False in newforms admin,django imagefield core false newforms admin tr...,django imagefield core false newforms admin tr...,https://stackoverflow.com/questions/34209,python|django|django-models,0.000291,"To get rid of ""This field is required,"" you ne...",0.180000,0.591429,django imagefield core false newforms admin
5,Django templates and variable attributes,django templates variable attributes using goo...,django templates variable attributes using goo...,https://stackoverflow.com/questions/35948,python|django|google-app-engine,0.005801,Or you can use the default django system which...,0.070463,0.578611,django templates variable attributes
6,Setup django with WSGI and apache,setup django wsgi apache sold mod_wsgi apache ...,setup django wsgi apache sold mod_wsgi apache ...,https://stackoverflow.com/questions/36806,python|django|apache|mod-wsgi,0.001668,What happens if you remove the directive?,0.071429,0.230952,setup django wsgi apache
7,How do I merge two dictionaries in a single ex...,merge two dictionaries single expression two p...,merge two dictionaries single expression two p...,https://stackoverflow.com/questions/38987,python|dictionary|merge,0.998339,"I wanted something similar, but with the abili...",0.009498,0.549654,merge two dictionaries single expression
8,How to know whether a window with a given titl...,know whether window given title already open t...,know whether window given title already open t...,https://stackoverflow.com/questions/45540,python|tkinter|tk,-0.000398,The previous answer works accordingly to the c...,0.206277,0.613853,know whether window given title already open tk
9,Django: Print url of view without hardcoding t...,django print url view without hardcoding url p...,django print url view without hardcoding url p...,https://stackoverflow.com/questions/47207,python|django,0.001324,"You can use , but that will only work for a pa...",0.210833,0.611111,django print url view without hardcoding url


In [3]:
# Import saved Wordvec Embeddings
import gensim
w2v_model = gensim.models.word2vec.Word2Vec.load('data/SO_word2vec_embeddings.bin')

**Create Sentence Embeddings**

In order to calculate the embeddings for an entire sentence, we defined the following function which averages the embeddings for each valid token

In [4]:
def question_to_vec(question, embeddings, dim=300):
    question_embedding = np.zeros(dim)
    valid_words = 0
    for word in question.split(' '):
        if word in embeddings:
            valid_words += 1
            question_embedding += embeddings[word]
    if valid_words > 0:
        return question_embedding/valid_words
    else:
        return question_embedding

In [5]:
all_title_embeddings = []
for title in data.processed_title:
    all_title_embeddings.append(question_to_vec(title, w2v_model))
all_title_embeddings = np.array(all_title_embeddings)

embeddings = pd.DataFrame(data = all_title_embeddings)
embeddings.to_csv('data/title_embeddings.csv', index=False)

e:\dev\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
e:\dev\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [24]:
all_title_embeddings = pd.read_csv('data/title_embeddings.csv').values

In [25]:
all_title_embeddings

array([[-0.04937908, -0.61464383,  1.14346249, ..., -0.79477589,
         0.53268059, -0.96669359],
       [ 1.19756782,  0.96940847, -0.29225311, ...,  1.43396515,
         0.84202259, -0.03741039],
       [-0.273733  ,  0.50807061, -0.87573742, ..., -0.22963291,
         1.36277719, -0.34878033],
       ...,
       [ 0.49968979,  0.81338476, -0.96863714, ..., -1.34989285,
         0.57098916, -1.68519899],
       [ 0.17646157,  0.61704408,  0.64064128, ...,  0.68547733,
        -0.45004148, -0.36241225],
       [ 0.4680537 ,  2.26560955, -0.12679541, ...,  0.0299069 ,
         0.8983882 , -0.10422469]])

In [7]:
import keras.backend as K

# Custom loss function to handle multilabel classification task
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

Using TensorFlow backend.


In [8]:
from keras.models import load_model
import keras.losses

keras.losses.multitask_loss = multitask_loss
model = load_model('data/Tag_predictor.h5')

In [11]:
def predict_tags(text, include_neutral=True):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)
    # Predict
    prediction = model.predict([x_test])[0]
    for i,value in enumerate(prediction):
        if value > 0.5:
            prediction[i] = 1
        else:
            prediction[i] = 0
    tags = tag_encoder.inverse_transform(np.array([prediction]))
    return tags

In [13]:
import re
import nltk
import inflect
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_SEQUENCE_LENGTH = 300
import pickle
with open('data/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

**Getting the most similar results**

So the way we actually calculate the the most similar results, is by comparing how far each result is from the query in terms of distance. This can only be done if both the query and the results are in a shared vector space. Fortunately, that is exactly what our word embeddings are for. They create each sentence as a vector in the embedding space, which makes it easy for us to distinguish them.

After we have those vectors, we can assign a Similarity Measure as a metric which measures the closeness of two vectors.

We use a custom similarity measure by incorporating the sentiment of the answers and the popularity of the votes in addition to cosine similarity.

In [29]:
from IPython.display import HTML
import logging
from sklearn.metrics.pairwise import cosine_similarity

search_string = "Combine lists of lists" 
search_string = ' '.join(normalize(tokenize_text(search_string)))
results_returned = "5" 
search_vect = np.array([question_to_vec(search_string, w2v_model)])    # Vectorize the user query

# Calculate Cosine similarites for the query and all titles
cosine_similarities = pd.Series(cosine_similarity(search_vect, all_title_embeddings)[0])

# Custom Similarity Measure
cosine_similarities = cosine_similarities*(1 + 0.4*data.overall_scores + 0.1*(data.sentiment_polarity))

output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
    output += '<a target="_blank" href='+ str(data.question_url[i])+'><h2>' + data.original_title[i] + '</h2></a>'
    output += '<h3> Similarity Score: ' + str(j) + '</h3>'
    output +='<p style="font-family:verdana; font-size:110%;"> '
    for i in data.question_content[i][:50].split():
        if i.lower() in search_string:
            output += " <b>"+str(i)+"</b>"
        else:
            output += " "+str(i)
    output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))

e:\dev\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
e:\dev\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
